<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/gw_agentic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: print hello world

print("hello world")

hello world


In [ ]:
!pip install google-generativeai tiktoken chromadb langgraph langchain langchain_community langchainhub ipykernel langchain_groq sentence_transformers boto3 langchain_aws docx exceptions
!pip install --upgrade python-docx

### Import dependencies

In [12]:
from typing import List, Any, Optional, TypedDict, Dict
from pydantic import BaseModel, Field, root_validator
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.schema import SystemMessage, HumanMessage
import boto3
from langchain_aws.chat_models.bedrock import ChatBedrock
from langgraph.graph import END, StateGraph, START
from docx import Document
import json

import re
from typing import Dict, List